In [2]:
!pip install osrm-py

  Preparing metadata (setup.py) ... done
  DEPRECATION: osrm-py is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for osrm-py ... done

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import requests
import pandas as pd

df_s_dp=pd.DataFrame()

sources_lat_list=[17.40713,17.46308	,17.53758,17.55849,17.5944,	17.59829,	17.57643,
                  	17.44174,	17.37449,	17.31975,	17.23785,	17.21239,	17.20388,	17.2038,	17.25335,	17.35022,]
source_long_list=[78.33044,	78.24922,	78.23448,	78.31313,	78.40774,	78.49089,	78.57166,	78.66665,	78.68084,
                  	78.6568,	78.58414,	78.52735,	78.47619,	78.40018,	78.37928,	78.36102]

# Demand points coordinates
demand_points = [
    (17.41343, 78.30131), (17.459217, 78.252203), (17.527048, 78.24003), (17.549523, 78.27773), (17.559485, 78.325715), (17.57153, 78.37946),
    (17.6066, 78.45457),(17.595073, 78.505587), (17.565378, 78.619475),
    (17.526233, 78.640257), (17.48407, 78.6661), (17.416915, 78.34454),
    (17.44637, 78.668743), (17.398865, 78.683545), (17.354695, 78.67091),(17.31261, 78.656528),
    (17.25649, 78.61857),(17.230818, 78.575885), (17.213877, 78.534427), (17.2051, 78.48928),
    (17.2034, 78.412433), (17.23794, 78.381955), (17.258953, 78.37756),(17.311622, 78.377375),
    (17.350922, 78.359623),(17.389173, 78.350535),
]

for i in range(len(source_long_list)):
    source_lat=sources_lat_list[i]
    source_lon=source_long_list[i]

    # Initialize an empty list to store distances
    distances = []

    # OSRM server URL
    osrm_url = "http://router.project-osrm.org/route/v1/driving/"

    # Iterate over demand points and calculate the distance to the source
    for point in demand_points:
        url = f"{osrm_url}{source_lon},{source_lat};{point[1]},{point[0]}?overview=false"
        response = requests.get(url)
        data = response.json()
        distance = data['routes'][0]['distance']/1000.0
        distances.append(distance)

    # Create a DataFrame with the distances
    df_distances = pd.DataFrame({
        'Demand_Point_LAT': [point[0] for point in demand_points],
        'Demand_Point_LON': [point[1] for point in demand_points],
        'Distance (Km)': distances
    })

    df_s_dp[f's{i}-dp']=distances


In [9]:
df_s_dp

,s0-dp,s1-dp,s2-dp,s3-dp,s4-dp,s5-dp,s6-dp,s7-dp,s8-dp,s9-dp,s10-dp,s11-dp,s12-dp,s13-dp,s14-dp,s15-dp
0,10.8108,8.8220,19.5082,37.5861,39.5745,48.5963,55.7604,47.4983,54.2737,46.6264,50.3870,43.2370,37.7022,45.7426,24.2659,12.0597
1,20.0741,0.7034,11.3896,29.4675,31.4559,40.4777,47.6418,56.7728,63.5482,55.9009,59.6503,52.5002,46.9654,55.0059,33.5292,21.3229
2,32.2096,12.8574,4.3878,25.4034,27.3919,36.4136,43.5777,66.4097,72.5249,58.0650,71.7858,64.6358,59.1009,67.1414,45.6647,33.4585
3,33.4606,14.1084,5.6388,16.0044,17.9929,27.0146,34.1787,57.0107,63.1259,71.1547,73.0368,65.8867,60.3519,68.3924,46.9157,34.7094
4,27.2876,18.8183,12.4110,10.3799,11.1290,21.6212,28.7853,51.6173,57.7325,65.7613,77.2671,70.5966,65.0618,73.1022,51.6256,39.4193
5,27.3465,26.6601,20.2529,9.5348,4.7191,16.2886,23.4527,46.2847,52.3999,60.4287,71.9345,61.1173,55.2498,63.2799,43.7673,31.5611
6,42.5886,44.4936,38.0863,27.3682,16.9742,5.8119,12.9760,35.8079,41.9231,49.9519,61.4577,59.6601,53.7926,61.8227,50.2191,43.7869
7,41.7897,41.0999,34.6926,23.9745,13.5805,2.4182,11.6283,34.4603,40.5755,48.6043,60.1101,57.7182,51.8506,59.8807,48.2771,41.8449
8,46.9283,53.7800,47.3728,36.6547,26.2606,17.9885,19.3528,18.0990,24.2142,32.2430,43.7488,50.5793,55.7722,63.7783,52.3646,45.5053
9,58.3837,57.6938,51.2865,40.5684,30.1744,21.9022,11.0958,35.7610,41.8762,49.9050,61.4108,68.2414,61.3980,69.4281,57.4297,50.9975


In [10]:
df_s_dp.to_csv("distance-matrix.csv")